In [1]:
import duckdb
import leafmap
import pandas as pd

In [2]:
con = duckdb.connect()
con.install_extension("httpfs")
con.load_extension("httpfs")
con.install_extension("spatial")
con.load_extension("spatial")

In [3]:
url = 'C:/Users/vance/Downloads/ukspills.xlsx'

In [4]:
con.sql("""
        CREATE TABLE IF NOT EXISTS spills as
        SELECT * FROM ST_Read('C:/Users/vance/Downloads/ukspills_clean.xlsx')
""")
con.table("spills")

┌──────────┬──────────────────────┬──────────────────────┬───┬──────────────────────┬────────────────────┐
│ OBJECTID │       COMPANY        │         SITE         │ … │     CONSTITUENCY     │ riverBasinDistrict │
│  int32   │       varchar        │       varchar        │   │       varchar        │      varchar       │
├──────────┼──────────────────────┼──────────────────────┼───┼──────────────────────┼────────────────────┤
│        1 │ Dwr Cymru Welsh Wa…  │ Trevethin CSO        │ … │ Torfaen              │ Western Wales      │
│        2 │ Dwr Cymru Welsh Wa…  │ Cwmbran Somerset R…  │ … │ Torfaen              │ Western Wales      │
│        3 │ Dwr Cymru Welsh Wa…  │ PONTNEWYNYDD CSO     │ … │ Torfaen              │ Western Wales      │
│        4 │ Dwr Cymru Welsh Wa…  │ Bridge St CSO        │ … │ Brecon and Radnors…  │ Western Wales      │
│        5 │ Dwr Cymru Welsh Wa…  │ Woodlands (Horrock…  │ … │ Newport West         │ Western Wales      │
│        6 │ Dwr Cymru Welsh Wa…  │ C

In [5]:
con.sql('''
    DESCRIBE spills
''')

┌────────────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│    column_name     │ column_type │  null   │   key   │ default │ extra │
│      varchar       │   varchar   │ varchar │ varchar │ varchar │ int32 │
├────────────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ OBJECTID           │ INTEGER     │ YES     │ NULL    │ NULL    │  NULL │
│ COMPANY            │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ SITE               │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ WATERBODY          │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ DURATION           │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ SPILL_NUMBER       │ INTEGER     │ YES     │ NULL    │ NULL    │  NULL │
│ Eastings           │ INTEGER     │ YES     │ NULL    │ NULL    │  NULL │
│ Northings          │ INTEGER     │ YES     │ NULL    │ NULL    │  NULL │
│ Latitude           │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ Longitude          │ DO

In [11]:
con.sql('''
    SELECT COMPANY, SUM(SPILL_NUMBER) AS NUMBER, ROUND(SUM(DURATION),1) AS SPILL_HOURS
    FROM spills
    GROUP BY COMPANY
    ORDER BY NUMBER DESC
''')

┌───────────────────────┬────────┬─────────────┐
│        COMPANY        │ NUMBER │ SPILL_HOURS │
│        varchar        │ int128 │   double    │
├───────────────────────┼────────┼─────────────┤
│ Dwr Cymru Welsh Water │  86277 │    602987.5 │
│ United Utilities      │  69245 │    425490.6 │
│ Yorkshire Water       │  54273 │    232053.6 │
│ Severn Trent Water    │  44765 │    249115.6 │
│ South West Water      │  37649 │    290271.3 │
│ Northumbrian Water    │  29697 │    107536.1 │
│ Wessex Water          │  21878 │    129957.4 │
│ Southern Water        │  16688 │    146819.6 │
│ Anglian Water         │  16082 │     89513.8 │
│ Thames Water          │   8014 │     74693.3 │
├───────────────────────┴────────┴─────────────┤
│ 10 rows                            3 columns │
└──────────────────────────────────────────────┘